<a href="https://colab.research.google.com/github/punkmic/Topic-Modeling-Classification/blob/master/MSDSTopicModel_Lab2_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## ⚡️ Make a Copy

Save a copy of this notebook in your Google Drive before continuing. Be sure to edit your own copy, not the original notebook.

In this lab, you will simply excute the code to create your classifications and your interactive LDAvis. Success in this lab depends primarily on having properly completed Homework 2.

Before continuing with this lab, be sure you have completed Homework 2 and saved your corpus and topic model to your Google Drive. They should be files in the root of your Google Drive called MSDS_HW2_corpus.p and MSDS_HW2_model.p. In this lab, you will load the model and use it for classification of documents in the corpus.

Mount your Google Drive via the left sidebar controls before continuing.

✏️  **Note:** You may need to restart the runtime after running the following imports/installs.

In [1]:
import numpy as np
import pickle

try:
  import pyLDAvis
except:
  !pip install pyLDAvis==2.1.2
  import pyLDAvis

try:
  import tmtoolkit
except:
  !pip install tmtoolkit
  import tmtoolkit

try:
  from lda import LDA
except: 
  !pip install lda
  from lda import LDA

from tmtoolkit.bow.bow_stats import doc_lengths
from tmtoolkit.topicmod.model_stats import generate_topic_labels_from_top_words
from tmtoolkit.topicmod.model_io import ldamodel_top_doc_topics
from tmtoolkit.topicmod.model_io import load_ldamodel_from_pickle
from tmtoolkit.topicmod.visualize import parameters_for_ldavis

/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable
/usr/local/lib/python3.7/dist-packages/past/builtins/misc.py:4: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Mapping


## Load the saved objects

Be sure that you have mounted your Google Drive and that the files are there.

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
with open("drive/MyDrive/MSDS_HW2_corpus.p", "rb") as corpusfile:
    corpus = pickle.load(corpusfile)

In [5]:
with open("drive/MyDrive/MSDS_HW2_model.p", "rb") as modelfile:
    model_info = load_ldamodel_from_pickle(modelfile)

model_info.keys()

dict_keys(['model', 'vocab', 'doc_labels', 'dtm'])

## Classification

As seen from the keys output above, the saved model is actually a dictionary containing the model itself, along with some other resources we will need.

Here, you will use those resources to create topic labels using [generate_topic_labels_from_top_words](https://tmtoolkit.readthedocs.io/en/latest/api.html#tmtoolkit.topicmod.model_stats.generate_topic_labels_from_top_words). Note that the vocabulary is provided as a list, but a numpy array is needed. Also note that the doc lengths are determined by the doc_lengths function using the saved dtm.

In [6]:
model = model_info["model"]
vocab = model_info["vocab"]
dtm = model_info["dtm"]
doc_labels = model_info["doc_labels"]

topic_labels = generate_topic_labels_from_top_words(
    model.topic_word_,
    model.doc_topic_,
    doc_lengths(dtm),
    np.array(vocab),
)

In [7]:
topic_labels

array(['1_shoe_good_fit_size', '2_shoe_nike_watch_great',
       '3_shoe_size_fit_buy', '4_shoe_watch_good_great',
       '5_shoe_great_wear_get', '6_shoe_great_run_love',
       '7_shoe_wear_buy_good', '8_shoe_good_great_wear',
       '9_shoe_fit_run_love', '10_shoe_good_great_comfortable'],
      dtype='<U30')

Document classifications can be retrieved via [ldamodel_top_doc_topics](https://tmtoolkit.readthedocs.io/en/latest/api.html?highlight=ldamodel_top_doc_topics#tmtoolkit.topicmod.model_io.ldamodel_top_doc_topics)

In [8]:
doc_topic = model.doc_topic_
documentclassifications = ldamodel_top_doc_topics(doc_topic, doc_labels, top_n=2, topic_labels=topic_labels) 

In [9]:
documentclassifications['rank_1']

document
0            4_shoe_watch_good_great (0.6197)
1               7_shoe_wear_buy_good (0.6651)
2               5_shoe_great_wear_get (0.445)
3            4_shoe_watch_good_great (0.7951)
4            4_shoe_watch_good_great (0.5917)
                         ...                 
21565         8_shoe_good_great_wear (0.6538)
21566          6_shoe_great_run_love (0.6622)
21567        2_shoe_nike_watch_great (0.7049)
21568    10_shoe_good_great_comfortable (0.9)
21569        4_shoe_watch_good_great (0.6538)
Name: rank_1, Length: 21570, dtype: object

### Attach the original corpus texts to the dataframe

Finally, include the corpus texts in the frame in order to see the text alongside the topics.

In [10]:
documentclassifications["text"] = corpus
documentclassifications['text']

document
0        the colour i received is not blue as shown but...
1        Very cute and is really practical. Fits better...
2        The watch was exactly what i ordered and I got...
3        This product came promptly and as described, p...
4        Why isn't Nike making these anymore?  I love t...
                               ...                        
21565                                              love it
21566            They fit perfect and the are really light
21567    The product was shipped in record speed! I lov...
21568    I like this item but they didn't have my size ...
21569                                            Love it !
Name: text, Length: 21570, dtype: object

## Visualization

[pyLDAvis](https://pyldavis.readthedocs.io/en/latest/readme.html) is a Python port of the LDAvis package in R, and is used as a tool for interpreting the topics in a topic model that has been fit to a corpus of text data. 

Execute the following code to create an interactive visualization of your topic model.

In [11]:
ldavis_params = parameters_for_ldavis(
    model.topic_word_,
    model.doc_topic_,
    dtm,
    vocab
)

In [ ]:
# joblib.externals.loky.process_executor.BrokenProcessPool -> https://github.com/bmabey/pyLDAvis/issues/144
!pip install --upgrade pandas==1.2

In [12]:
%matplotlib inline
vis = pyLDAvis.prepare(**ldavis_params)
pyLDAvis.enable_notebook(local=True)
pyLDAvis.display(vis)

/usr/local/lib/python3.7/dist-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated since IPython 4.0. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


⚡️ **For a better screenshot:**

 * Select the code cell above which contains the display code
 * Select the 3 dots at the far right for "More cell actions"
 * Select "View output fullscreen"



## Submit a screenshot for peer evaluation

Once you have completed execution of the above code, and have generated the visualization of your topic model, submit a screenshot of your interactive LDAvis above as proof of completion of the lab. This should be submitted as an image file in the peer review assessment after Lab 2 in Coursera.

💡 Tip: The LDAvis tends to be a bit more readable in Colab's light theme than it is in the dark theme (Go to Tools > Settings > Sight > Theme > light). Also, it may be easier to get a good screenshot by viewing the chart at full screen (see the ⚡️ tip above).